# Backdoor Unlearning

## Outline

1. Experimental setup (generating configs)
2. Clean model training
3. Poisoned model training
4. First-order unlearning
5. Second-order unlearning
6. Visualizing results


## Experimental Setup

- All configurations to test are defined in the `[train|poison|unlearn].json` files (see below).
- If parameters are passed as list, all their combinations are tested in a grid-search manner.
- Only a single combination is provided for this demo. The original combinations are in `Applications/Poisoning/configs`
- The function generates directories and configuration files for each combination. They are later used by an evaluation script to run the experiment. This allows for parallelization and distributed execution.

In [1]:
import sys
sys.path.append('../')


In [2]:
# only if you are using CUDA devices
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [3]:
from conf import BASE_DIR
from Applications.Poisoning.gen_configs import main as gen_configs

model_folder = BASE_DIR/'models'/'poisoning'
train_conf = BASE_DIR/'Applications'/'Poisoning'/'configs'/'demo'/'train.json'
poison_conf = BASE_DIR/'Applications'/'Poisoning'/'configs'/'demo'/'poison.json'
unlearn_conf = BASE_DIR/'Applications'/'Poisoning'/'configs'/'demo'/'unlearn.json'

gen_configs(model_folder, train_conf, poison_conf, unlearn_conf)

In [4]:
from Applications.Poisoning.poison.poison_models import train_poisoned
from Applications.Poisoning.configs.demo.config import Config

poisoned_folder = model_folder/'budget-10000'/'seed-42'
clean_folder = model_folder/'clean'
first_unlearn_folder = model_folder/'budget-10000'/'seed-42'/'first-order'
second_unlearn_folder = model_folder/'budget-10000'/'seed-42'/'second-order'


poison_kwargs = Config.from_json(poisoned_folder/'poison_config.json')
train_kwargs = Config.from_json(poisoned_folder/'train_config.json')


2024-07-11 18:41:29.988905: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda-12.2/lib64
2024-07-11 18:41:29.988921: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Clean Model Training

- Train a clean model for reference.

## Train Poisoned Model

- Select one of the generated configurations and train a poisoned model.
- The poisoning uses an `injector` object which can be persisted for reproducibility. It will inject the backdoors/label noise into the same samples according to a seed. In our experiments, we worked with label noise poisoning.

In [5]:
from Applications.Poisoning.poison.poison_models import train_poisoned
from Applications.Poisoning.configs.demo.config import Config

poisoned_folder = model_folder/'budget-10000'/'seed-42'
clean_folder = model_folder/'clean'
first_unlearn_folder = model_folder/'budget-10000'/'seed-42'/'first-order'
second_unlearn_folder = model_folder/'budget-10000'/'seed-42'/'second-order'


poison_kwargs = Config.from_json(poisoned_folder/'poison_config.json')
train_kwargs = Config.from_json(poisoned_folder/'train_config.json')



In [6]:
poisoned_weights = poisoned_folder/'best_model.hdf5'       # model that has been trained on poisoned data
fo_repaired_weights = poisoned_folder/'fo_repaired.hdf5'   # model weights after unlearning (first-order)
so_repaired_weights = poisoned_folder/'so_repaired.hdf5'   # model weights after unlearning (second-order)
injector_path = poisoned_folder/'injector.pkl'             # cached injector for reproducibility
clean_results = model_folder/'clean'/'train_results.json'  # path to reference results on clean dataset


## Unlearning

- Perform the first-order and second-order unlearning. The unlearning is wrapped in a function that
    - loads the clean data, saves the original labels
    - injects the poison (label noise)
    - creates difference set Z using `injector.injected_idx`
    - main unlearning happens in `Applications.Poisoning.unlearn.common.py:unlearn_update` and the thereby called `iter_approx_retraining` method
- The variable naming follows the following ideas:
    - `z_x`, `z_y`: features (x) and labels (y) in set `Z`
    - `z_x_delta`, `z_y_delta`: changed features and labels (`z_x == z_x_delta` here and `z_y_delta` contains the original (fixed) labels)
- A word about why iterative:
    - The approximate retraining is configured to unlearn the desired changes in one step.
    - To avoid putting a lot of redundant erroneous samples in the changing set `Z`, the iterative version
        - takes a sub-sample (`prio_idx`) of `hvp_batch_size` in the delta set `Z`
        - makes one unlearning step
        - recalculates the delta set and focuses only on remaining errors
    - The idea here is that similar to learning, it is better to work iteratively in batches since the approximation quality of the inverse hessian matrix decreases with the number of samples included (and the step size)

In [7]:
from Applications.Poisoning.unlearn.first_order import run_experiment as fo_experiment
from Applications.Poisoning.unlearn.second_order import run_experiment as so_experiment

fo_unlearn_kwargs = Config.from_json(poisoned_folder/'first-order'/'unlearn_config.json')
so_unlearn_kwargs = Config.from_json(poisoned_folder/'second-order'/'unlearn_config.json')


In [8]:
from Applications.Poisoning.train import main as train
from Applications.Poisoning.evaluate import evaluate

# train one clean and one poisoned model
# datasets = ['Cifar10', 'Cifar100', 'SVHN', 'FashionMnist']
datasets = ['Cifar100', 'Cifar10', 'SVHN']
# modelnames = ['extractfeatures_VGG16', 'classifier_VGG16']
modelnames = ['VGG16', 'RESNET50']

In [9]:
import json
import os

results = {
    'clean': {},
    'poisoned': {},
    'first_order_unlearning': {},
    'second_order_unlearning': {}
}

update_targets = ['classifier', 'feature_extractor']

for dataset in datasets:
    results['clean'][dataset] = {}
    results['poisoned'][dataset] = {}
    results['first_order_unlearning'][dataset] = {}
    results['second_order_unlearning'][dataset] = {}
   
    print('#' * 60)
    print(f" UNLEARNING ")
    print('#' * 60)
    print('\n\n')

    for modelname in modelnames:
        for update_target in update_targets:
            print(f"* Evaluating {modelname} on {dataset} poisoned model *")
            poisoned_accuracy = evaluate(model_folder=poisoned_folder, dataset=dataset, modelname=modelname, type='poisoned')
            results['poisoned'][dataset][modelname] = poisoned_accuracy
            
            print(f"* First-order unlearning {modelname} on {dataset} poisoned model *")
            fo_experiment(poisoned_folder/'first-order', train_kwargs, poison_kwargs, fo_unlearn_kwargs, dataset=dataset, modelname=modelname, update_target=update_target)
            print(f"* Evaluating {modelname} on {dataset} after first-order unlearning *")
            fo_repaired_accuracy = evaluate(model_folder=first_unlearn_folder, dataset=dataset, modelname=modelname, type='repaired')
            results['first_order_unlearning'][dataset][modelname] = fo_repaired_accuracy
        

            print(f"* Second-order unlearning {modelname} on {dataset} poisoned model *")
            so_experiment(poisoned_folder/'second-order', train_kwargs, poison_kwargs, so_unlearn_kwargs, dataset=dataset, modelname=modelname, update_target=update_target)
            print(f"* Evaluating {modelname} on {dataset} after second-order unlearning *")
            so_repaired_accuracy = evaluate(model_folder=second_unlearn_folder, dataset=dataset, modelname=modelname, type='repaired')
            results['second_order_unlearning'][dataset][modelname] = so_repaired_accuracy
        

############################################################
 UNLEARNING 
############################################################



* Evaluating VGG16 on Cifar100 poisoned model *


2024-07-11 18:41:35.433048: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-07-11 18:41:35.433065: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: conite-HP
2024-07-11 18:41:35.433069: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: conite-HP
2024-07-11 18:41:35.433153: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 555.42.2
2024-07-11 18:41:35.433164: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 555.42.2
2024-07-11 18:41:35.433167: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 555.42.2
2024-07-11 18:41:35.433343: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical 

Loading weights from None
Model weights loaded successfully from /home/conite/Documents/WORKSPACE/PROJECTS/Memoire M2/code/MachineUnlearningUpdate/models/poisoning/budget-10000/seed-42/Cifar100_VGG16_poisoned_model.hdf5.
Accuracy Cifar100 model: 0.6082
* First-order unlearning VGG16 on Cifar100 poisoned model *
Loading weights from None


2024-07-11 18:41:44.928052: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.
2024-07-11 18:41:45.192172: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.
2024-07-11 18:41:45.252326: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.
2024-07-11 18:41:45.315803: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.
2024-07-11 18:41:45.389537: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.


(3, 3, 3, 64)
(64,)
(64,)
(64,)
(3, 3, 64, 64)
(64,)
(64,)
(64,)
(3, 3, 64, 128)
(128,)
(128,)
(128,)
(3, 3, 128, 128)
(128,)
(128,)
(128,)
(3, 3, 128, 256)
(256,)
(256,)
(256,)
(3, 3, 256, 256)
(256,)
(256,)
(256,)
(3, 3, 256, 256)
(256,)
(256,)
(256,)
(4096, 4096)
(4096,)
(4096,)
(4096,)
(4096, 4096)
(4096,)
(4096,)
(4096,)
(4096, 100)
(100,)
Updated 38 weights. 0 weights were not updated.
Updated non-trainable weights: [(28, 'batch_normalization_4/moving_mean:0'), (29, 'batch_normalization_4/moving_variance:0'), (30, 'conv2d_5/kernel:0'), (31, 'conv2d_5/bias:0'), (32, 'batch_normalization_5/gamma:0'), (33, 'batch_normalization_5/beta:0'), (34, 'batch_normalization_5/moving_mean:0'), (35, 'batch_normalization_5/moving_variance:0'), (36, 'conv2d_6/kernel:0'), (37, 'conv2d_6/bias:0'), (38, 'batch_normalization_6/gamma:0'), (39, 'batch_normalization_6/beta:0'), (40, 'batch_normalization_6/moving_mean:0'), (41, 'batch_normalization_6/moving_variance:0'), (42, 'dense/kernel:0'), (43, 'den

../Applications/Poisoning/unlearn/common.py:210: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=(10, 10))


Early stopping at iteration 20. Update norm 1.675372601765993e+21 > [26590300868.25586, 20]
(3, 3, 3, 64)
(64,)
(64,)
(64,)
(3, 3, 64, 64)
(64,)
(64,)
(64,)
(3, 3, 64, 128)
(128,)
(128,)
(128,)
(3, 3, 128, 128)
(128,)
(128,)
(128,)
(3, 3, 128, 256)
(256,)
(256,)
(256,)
(3, 3, 256, 256)
(256,)
(256,)
(256,)
(3, 3, 256, 256)
(256,)
(256,)
(256,)
(4096, 4096)
(4096,)
(4096,)
(4096,)
(4096, 4096)
(4096,)
(4096,)
(4096,)
(4096, 100)
(100,)
Updated 38 weights. 0 weights were not updated.
Updated non-trainable weights: [(28, 'batch_normalization_4/moving_mean:0'), (29, 'batch_normalization_4/moving_variance:0'), (30, 'conv2d_5/kernel:0'), (31, 'conv2d_5/bias:0'), (32, 'batch_normalization_5/gamma:0'), (33, 'batch_normalization_5/beta:0'), (34, 'batch_normalization_5/moving_mean:0'), (35, 'batch_normalization_5/moving_variance:0'), (36, 'conv2d_6/kernel:0'), (37, 'conv2d_6/bias:0'), (38, 'batch_normalization_6/gamma:0'), (39, 'batch_normalization_6/beta:0'), (40, 'batch_normalization_6/moving_

## FORGET SCORE

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, accuracy_score
from torch.utils.data import DataLoader
import tensorflow as tf
from tensorflow.keras.models import load_model

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def calculate_confidence(model, x_data):
    y_pred = model.predict(x_data)
    probs = tf.nn.softmax(y_pred, axis=1).numpy()
    max_probs = np.max(probs, axis=1)
    return max_probs


def calculate_confusion_matrix(model, x_data, y_true):
    y_pred = model.predict(x_data)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_true, axis=1) if len(y_true.shape) > 1 else y_true
    cm = confusion_matrix(y_true_classes, y_pred_classes)

    accuracy = np.trace(cm) / np.sum(cm)
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
    else:
        tn, fp, fn, tp = 0, 0, 0, 0
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                if i == j:
                    if i == 1:
                        tp = cm[i, j]
                    else:
                        tn += cm[i, j]
                else:
                    if i == 1:
                        fn += cm[i, j]
                    else:
                        fp += cm[i, j]
    return tn, fp, fn, tp

def calculate_forget_score(tn_before, fp_before, fn_before, tp_before, tn_after, fp_after, fn_after, tp_after):
    delta = 0.01
    tpr_before = tp_before / (tp_before + fn_before + delta)
    fpr_before = fp_before / (fp_before + tn_before + delta)
    tpr_after = tp_after / (tp_after + fn_after + delta)
    fpr_after = fp_after / (fp_after + tn_after + delta)

    epsilon = np.nanmax([
        np.log(1 - delta - fpr_after) - np.log(tpr_after),
        np.log(1 - delta - fn_after) - np.log(tpr_after),
        np.log(1 - delta - fpr_before) - np.log(tpr_before),
        np.log(1 - delta - fn_before) - np.log(tpr_before)
    ])

    return epsilon


def plot_confusion_matrix(y_true, y_pred, title):
    if len(y_true.shape) > 1:
        y_true = np.argmax(y_true, axis=1)
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


def evaluate_model_accuracy(model, x_test, y_test):
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1) if len(y_test.shape) > 1 else y_test
    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    return accuracy

def load_and_evaluate_models(datasets, models, clean_folder, unlearn_folder):
    results = {}
    
    for dataset_name, dataset in datasets.items():
        results[dataset_name] = {}
        
        # Load the dataset
        (x_train, y_train), (x_test, y_test), (x_valid, y_valid) = dataset.load()
        
        for model_name in models[dataset_name]:
            model_fn = models[dataset_name][model_name]
            results[dataset_name][model_name] = {}
            print(f"Evaluating {model_name} on {dataset_name}")
            
            try:
                # Load the clean model
                model_clean = model_fn()
                model_clean.load_weights(clean_folder / f'{model_name}_best_model.hdf5')

                # Load the unlearned model
                model_unlearned = model_fn()
                model_unlearned.load_weights(unlearn_folder / f'{model_name}_repaired_model.hdf5')
            except:
                print(f"Error loading models for {model_name} on {dataset_name}")
                continue
            # Evaluate the accuracy of the models
            accuracy_clean = evaluate_model_accuracy(model_clean, x_test, y_test)
            accuracy_unlearned = evaluate_model_accuracy(model_unlearned, x_test, y_test)
            print(f"Accuracy of the clean model: {accuracy_clean:.4f}")
            print(f"Accuracy of the unlearned model: {accuracy_unlearned:.4f}")

            # Compute confusion matrix for clean model
            y_pred_clean = model_clean.predict(x_test).argmax(axis=1)
            #plot_confusion_matrix(y_test, y_pred_clean, f'{model_name} Clean Model Confusion Matrix')

            # Compute confusion matrix for unlearned model
            y_pred_unlearned = model_unlearned.predict(x_test).argmax(axis=1)
            #plot_confusion_matrix(y_test, y_pred_unlearned, f'{model_name} Unlearned Model Confusion Matrix')


            # Compute confidence and confusion matrix for clean model
            clean_confidences = calculate_confidence(model_clean, x_test)
            tn_clean, fp_clean, fn_clean, tp_clean = calculate_confusion_matrix(model_clean, x_test, y_test)

            # Compute confidence and confusion matrix for unlearned model
            unlearning_confidences = calculate_confidence(model_unlearned, x_test)
            tn_unlearned, fp_unlearned, fn_unlearned, tp_unlearned = calculate_confusion_matrix(model_unlearned, x_test, y_test)

            # Calculate forget score
            forget_score = calculate_forget_score(tn_clean, fp_clean, fn_clean, tp_clean, tn_unlearned, fp_unlearned, fn_unlearned, tp_unlearned)
            print(f"Forget Score for {model_name} on {dataset_name}: {forget_score:.4f}")

            results[dataset_name][model_name] = {
                'clean_accuracy': accuracy_clean,
                'unlearned_accuracy': accuracy_unlearned,
                'forget_score': forget_score
            }

    return results




**Evaluation**

In [ ]:
import sys
sys.path.append('../')
# import TensorDataset
from torch.utils.data import DataLoader, TensorDataset
import os
from conf import BASE_DIR
from Applications.Poisoning.gen_configs import main as gen_configs
from Applications.Poisoning.model import extractfeatures_VGG16, classifier_VGG16, extractfeatures_RESNET50, classifier_RESNET50, get_VGG16_CIFAR100, get_VGG16_CIFAR10, get_VGG16_SVHN, get_RESNET50_CIFAR100, get_RESNET50_CIFAR10, get_RESNET50_SVHN, extractfeatures_RESNET50_CIFAR100, extractfeatures_VGG16_CIFAR100, classifier_RESNET50_CIFAR100, classifier_VGG16_CIFAR100
from Applications.Poisoning.dataset import Cifar10, SVHN, FashionMnist, Cifar100


model_folder = BASE_DIR/'models'/'poisoning'

datasets = {
    'Cifar10': Cifar10,
    'SVHN': SVHN,
    'Cifar100': Cifar100
}

models = {
    'Cifar10': {
        'Cifar10_VGG16': get_VGG16_CIFAR10,
        'Cifar10_RESNET50': get_RESNET50_CIFAR10,
    },
    'SVHN': {
        'SVHN_VGG16': get_VGG16_SVHN,
        'SVHN_RESNET50': get_RESNET50_SVHN,
    },
    'Cifar100': {
        'Cifar100_VGG16': get_VGG16_CIFAR100,
        'Cifar100_RESNET50': get_RESNET50_CIFAR100,
    }
}


clean_folder = model_folder/'clean'
poisoned_folder = model_folder/'budget-10000'/'seed-42'
first_unlearn_folder = model_folder/'budget-10000'/'seed-42'/'first-order'
second_unlearn_folder = model_folder/'budget-10000'/'seed-42'/'second-order'

# Run evaluation with load_and_evaluate_models
results = load_and_evaluate_models(datasets, models, clean_folder, first_unlearn_folder)

# Print final results
for dataset_name, dataset_results in results.items():
    for model_name, model_results in dataset_results.items():
        print(f"{dataset_name} - {model_name}:")
        print(f"  Clean Accuracy: {model_results['clean_accuracy']:.4f}")
        print(f"  Unlearned Accuracy: {model_results['unlearned_accuracy']:.4f}")
        print(f"  Forget Score: {model_results['forget_score']:.4f}")

Evaluating Cifar10_VGG16 on Cifar10
Loading weights from None
Error loading models for Cifar10_VGG16 on Cifar10
Evaluating Cifar10_RESNET50 on Cifar10
Loading weights from None
Error loading models for Cifar10_RESNET50 on Cifar10
Evaluating SVHN_VGG16 on SVHN
Loading weights from None
Error loading models for SVHN_VGG16 on SVHN
Evaluating SVHN_RESNET50 on SVHN
Loading weights from None
Error loading models for SVHN_RESNET50 on SVHN
Evaluating Cifar100_VGG16 on Cifar100
Loading weights from None
Error loading models for Cifar100_VGG16 on Cifar100
Evaluating Cifar100_RESNET50 on Cifar100
Loading weights from None
Error loading models for Cifar100_RESNET50 on Cifar100
Cifar10 - Cifar10_VGG16:


KeyError: 'clean_accuracy'

In [ ]:
# import json
# import os

# results = {
#     'clean': {},
#     'poisoned': {},
#     'first_order_unlearning': {},
#     'second_order_unlearning': {}
# }

# update_targets = ['feature_extractor', 'classifier']
# for dataset in datasets:
#     for modelname in modelnames:
#         for update_target in update_targets:
#             print(f"* First-order unlearning {modelname} on {dataset} poisoned model *")
#             fo_experiment(poisoned_folder/'first-order', train_kwargs, poison_kwargs, fo_unlearn_kwargs, dataset=dataset, modelname=modelname, update_target=update_target)
#             print(f" * Second-order unlearning {modelname} on {dataset} poisoned model *")  
#             so_experiment(poisoned_folder/'second-order', train_kwargs, poison_kwargs, so_unlearn_kwargs, dataset=dataset, modelname=modelname, update_target=update_target)
